# LLM text generation notebook for Google Colab

This notebook uses [https://github.com/oobabooga/text-generation-webui](https://github.com/oobabooga/text-generation-webui) to run the [Pygmalion](https://huggingface.co/models?search=pygmalion) conversational models (NSFW) in chat mode.

Run all the cells and a public gradio URL will appear at the bottom in around 5 minutes.

https://status.gradio.app/

## Parameters

* **save_logs_to_google_drive**: saves your chat logs, characters, and softprompts to Google Drive automatically, so that they will persist across sessions.
* **text_streaming**: streams the text output in real time instead of waiting for the full response to be completed.
](https://oobabooga.github.io/silero-samples/).
* **activate_sending_pictures**: adds a menu for sending pictures to the bot, which are automatically captioned using BLIP.
* **activate_character_bias**: an extension that adds an user-defined, hidden string at the beginning of the bot's reply with the goal of biasing the rest of the response.
* **chat_language**: if different than English, activates automatic translation using Google Translate, allowing you to communicate with the bot in a different language.

## Characters

You can use the following websites to create characters compatible with this web UI:

* [JSON character creator](https://oobabooga.github.io/character-creator.html)
* [AI Character Editor](https://zoltanai.github.io/character-editor/)

## Credits

Based on the [original notebook by 81300](https://colab.research.google.com/github/81300/AI-Notebooks/blob/main/Colab-TextGen-GPU.ipynb).
# -------------------------------------
***Reminder:*** *if the notebook colab is not working in out of no where for some reason, then i'm probably doing something like adding stuff, removing stuff etc..or it's just colab being weird again*


###**⚠️DO NOT USE FOR NOW BECAUSE I AM CURRENTLY MERGING SILLYTAVERN⚠️ [05/20/2023]**
+ Added the **NEW** Pygmalion Models, these are the **Pygmalion-13B** and **Metharme-13B**

+ Replaced Gpt 4 x alpaca with Wizard-Vicuna-13B-Uncensored-GPTQ

+ ~~Reverted Ooba back to the latest version~~ Nevermind, everything is not working for some reason 

+ Added PygmalionCoT-7B


#####**Things needed to be done:**
+ Add AutoGPT (Destroy Humanity)
+ Fix EdgeGPT extension (Bots can use Bing AI Search & GoogleWordCloud)
+ Add superbooga (Infinite pseudo context)
+ Add AnnoyLTM (Fking finally Bots are not a dementia patient anymore wooo!!)
+ Add KoboldAI


#**Some Information About The Models**
**WizardVicuna 13B**
+ A **NEW** [WizardLM](https://github.com/nlpxucan/WizardLM) model that was trained with extra steps; Wizard's dataset + ChatGPT's conversation extension + Vicuna's tuning method, though its not entirely a merged model between WizardLM and VicunaLM but rather, the model used the same fine tuning method as Vicuna.

+ The model's benchmark is approximately (￢_￢) 3% behind GPT 3.5 and 7% better than Vicuna 13B model and 9% better than WizardLM 7B, for more detailed information. Click this [Github Page](https://github.com/melodysdreamj/WizardVicunaLM)

+ A bit disappointing to be honest....i assume that the 4bit quantization reduce it's quality response in exchange for faster response time, I would rather use PygmalionCot 7B instead.

**Pygmalion 13B**
+ Pygmalion 13B is a dialogue model based on Meta's LLaMA-13B, meaning it was trained using Llama-13B as the base model and has been fine-tuned using a subset of the data from Pygmalion-6B-v8-pt4
+ It is Currently **VERSION 1**
+ Reminder: the quality response might differ because this colab notebook used the 4bit version.

**Metharme 13B**
+ Upgraded version of Metharme 7B model

**PygmalionCoT 7B**
+ A merged model between Pygmalion 7B and [And kaiokendev's 7b SuperCOT-LoRA (Chain of thought)](https://huggingface.co/kaiokendev/SuperCOT-LoRA)
+ The model is much more coherent than Pygmalion 7B because of the LoRa that was merged, the LoRa was trained for Llama based models to improve instructions comprehension of bots.
+Tried it and it's pretty good, however it's NSFW factor is a bit toned down but overall the bots are more in character which is an improvement.

**Pygmalion 7B**
+ Pygmalion model that has the same architecture as a Llama model
+ Better Convo, verbose
+ Better Roleplaying 
+ However a bit difficult in initiating NSFW RP and might resorted to roasting you if you forced the bot to do it.

**Metharme 7B**
+ A New [EXPERIMENTAL] model that was trained by the pygmalion devs for storywriting, RPG text simulator, roleplay, etc..
+ Seems ok

**Pygmalion 350M (CPU)**
+ Don't even bother using this, it's mainly for debug stuff because of its small size, it is a very convenient model for doing maintenance stuff.

**Pygmalion 6B main (Outdated)**
+ The OG Pygmalion Model that specialized in NSFW roleplay

**Pygmalion 6B dev (Outdated)**
+ Not really that much different when compared to either the main or the original branch.

**Pygmalion 6B original (Outdated)**
+ The OG OG Pygmalion Model that specialized in NSFW roleplay and it's a bit too overtrained, that resulted in outputting a lengthy response

In [ ]:
#@title Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }
# Animations ↓↓
# &nbsp; &nbsp;<img src="https://huggingface.co/Imablank/AnythingV3-1/resolve/main/ooba/game-start.gif?size=90" width="90"/> 

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#@title ##**🚀Start TextGenWebUI🪶**
from IPython.display import clear_output, display, HTML
from IPython.utils import capture
from google.colab import files
from PIL import Image
import os, sys, base64, subprocess, json, shutil, requests, time, pathlib

#PARAMS
#@markdown ####**📖Setup Your UserProfile**

UserName = "" #@param{type:'string'}
Upload_ProfilePic = False #@param{type:'boolean'}
save_logs_to_google_drive = False #@param {type:"boolean"}

#@markdown ######<font color=gray>*Save your data such as your name, logs, characters, etc.. in google drive*
#@markdown ---
#@markdown ####**⚙️Configure and Run🚀**
model = "PygmalionCoT-7b" #@param ["Pygmalion-13B-4bit", "Metharme-13B-4bit", "PygmalionCoT-7b", "Pygmalion-7B", "Metharme-7B", "Wizard-Vicuna-13B-Uncensored-GPTQ", "Pygmalion-350m(CPU)", "Pygmalion_6B_main_Sharded", "Pygmalion_6B_original_Sharded", "Pygmalion_6B_dev_Sharded"]
text_streaming = False #@param {type:"boolean"}
activate_sending_pictures = False #@param {type:"boolean"}
activate_character_bias = False #@param {type:"boolean"}

GrantBot_Access_to_Bing = False #param {type:"boolean"}

chat_language = "English" #@param ['Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Azerbaijani', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Bulgarian', 'Catalan', 'Cebuano', 'Chinese (Simplified)', 'Chinese (Traditional)', 'Corsican', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Esperanto', 'Estonian', 'Finnish', 'French', 'Frisian', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hmong', 'Hungarian', 'Icelandic', 'Igbo', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Kurdish', 'Kyrgyz', 'Lao', 'Latin', 'Latvian', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Mongolian', 'Myanmar (Burmese)', 'Nepali', 'Norwegian', 'Nyanja (Chichewa)', 'Pashto', 'Persian', 'Polish', 'Portuguese (Portugal, Brazil)', 'Punjabi', 'Romanian', 'Russian', 'Samoan', 'Scots Gaelic', 'Serbian', 'Sesotho', 'Shona', 'Sindhi', 'Sinhala (Sinhalese)', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog (Filipino)', 'Tajik', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Welsh', 'Xhosa', 'Yiddish', 'Yoruba', 'Zulu']
activate_google_translate = (chat_language != "English")
RunWebUI = True #@param {type:"boolean"}
GetAPI = False #@param {type:"boolean"}
#@markdown > <font color=#eab676>Run SillyTavern instead of gradio or Get the oobabooga public API .
Debug = False #@param{type:'boolean'}
#@markdown > <font color=gray> *no bocchi? :(*

#@markdown &nbsp; &nbsp;<img src="https://huggingface.co/Imablank/AnythingV3-1/resolve/main/ooba/bocchi-the-rock-crying.gif?size=90" width="90"/> 

ot=" "
load_in_8bit = True
if "Wizard-Vicuna-13B-Uncensored-GPTQ" in model or "4bit" in model: load_in_8bit = False
old_ooba = True
kobold_memory = True

#Main Directories:
base_folder = "/content/drive/MyDrive/oobabooga-data"
main_dir = "/content/text-generation-webui/"
cache = f"{main_dir}cache"

#PART 1 INSTALL TextGeN
if save_logs_to_google_drive:
  from google.colab import drive
  drive.mount('/content/drive')

def install_silly():
  ForceInitSteps = []
  class IncrementialInstall:
    def __init__(self, root = "/", tasks = [], force = []):
        self.tasks = tasks
        self.path = os.path.join(root, ".ii")
        self.completed = list(filter(lambda x: not x in force, self.__completed()))

    def __completed(self):
        try:
            with open(self.path) as f:
                return json.load(f)
        except:
            return []

    def addTask(self, name, func):
        self.tasks.append({"name": name, "func": func})

    def run(self):
        todo = list(filter(lambda x: not x["name"] in self.completed, self.tasks))
        try:
            for task in todo:
                task["func"]()
                self.completed.append(task["name"])
        finally:
            with open(self.path, "w") as f:
                json.dump(self.completed, f)

  def create_paths(paths):
    for directory in paths:
        if not os.path.exists(directory):
            os.makedirs(directory)

  def link(srcDir, destDir, files):
    '''
        Link source to dest copying dest to source if not present first
    '''
    for file in files:
        source = os.path.join(srcDir, file)
        dest = os.path.join(destDir, file)
        if not os.path.exists(source):
            !cp -r "$dest" "$source"
        !rm -rf "$dest"
        !ln -fs "$source" "$dest"
  from google.colab import drive
  if not save_logs_to_google_drive:
    create_paths([
            "/content/SillyTavern-Data"
    ])
  ii = IncrementialInstall(force=ForceInitSteps)
# ---
# SillyTavern py modules
  def cloneTavern():
    %cd /
    !git clone https://github.com/Cohee1207/SillyTavern
    !wget https://huggingface.co/Imablank/AnythingV3-1/resolve/main/ooba/globals.py && wget https://huggingface.co/Imablank/AnythingV3-1/resolve/main/ooba/extras_server.py 
  ii.addTask("Clone SillyTavern", cloneTavern)
  ii.run()
  kargs = ["/content/ckds"]
  kargs += ["--localtunnel", "yes"]

# ---
# nodejs
  %cd /
  def installNode():
    !npm install -g n
    !n 19
    !node --version
  installNode()
# ---
# TavernAI extras
  import globals
  globals.params = []
  globals.params.append('--cpu')
  %cd /SillyTavern
  if save_logs_to_google_drive:
    %env googledrive=2
    def setupTavernPaths():
        %cd /SillyTavern
        tdrive = "/content/drive/MyDrive/SillyTavern"
        create_paths([
                tdrive,
                os.path.join("public", "groups"),
                os.path.join("public", "group chats")
        ])
        link(tdrive, "public", [
                "settings.json",
                "backgrounds",
                "characters",
                "chats",
                "User Avatars",
                "worlds",
                "group chats",
                "groups",
        ])
    ii.addTask("Setup Tavern Paths", setupTavernPaths)
  def installTavernDependencies():
    %cd /SillyTavern
    !npm install
    !npm install -g localtunnel
  ii.addTask("Install Tavern Dependencies", installTavernDependencies)
  ii.run()

def universal_download(link1, path1, nan3):
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link1} -d {path1} -o {nan3}

def edgegpt():
  import zipfile
  %cd /content/
  !wget https://huggingface.co/Imablank/AnythingV3-1/resolve/main/ooba/EdgeGPT_working.zip
  with zipfile.ZipFile('/content/EdgeGPT_working.zip', 'r') as zip:
    zip.extractall("/content/text-generation-webui/extensions")
  !pip install -r {main_dir}extensions/EdgeGPT/requirements.txt && rm -rf /content/EdgeGPT_working.zip
  %cd {main_dir}

def install_ooba():
  !apt-get -y install -qq aria2
     
  !mkdir {main_dir}cache
  if save_logs_to_google_drive:
    if not os.path.exists(f"{base_folder}"):
      os.mkdir(f"{base_folder}")
    if not os.path.exists(f"{base_folder}/logs"):
      os.mkdir(f"{base_folder}/logs")
    if not os.path.exists(f"{base_folder}/User"):
      os.mkdir(f"{base_folder}/User")
    if not os.path.exists(f"{base_folder}/softprompts"):
      os.mkdir(f"{base_folder}/softprompts")
    if not os.path.exists(f"{base_folder}/characters"):
      shutil.move("text-generation-webui/characters", f"{base_folder}/characters")
    else:
      !rm -r "text-generation-webui/characters"
    
    !rm -r "text-generation-webui/softprompts"
    !ln -s "$base_folder/logs" "text-generation-webui/"
    !ln -s "$base_folder/softprompts" "text-generation-webui/softprompts"
    !ln -s "$base_folder/characters" "text-generation-webui/characters"
    !ln -s "$base_folder/User" .
  else:
    !mkdir text-generation-webui/logs
    
  !ln -s text-generation-webui/logs .
  !ln -s text-generation-webui/characters .
  !ln -s text-generation-webui/models .
    
  %rm -r sample_data
  %cd {main_dir}
  !wget https://oobabooga.github.io/settings-colab.json -O settings-colab-template.json

  !rm -rf "/content/text-generation-webui/presets/Default.txt" "/content/text-generation-webui/settings-colab-template.json" "/content/text-generation-webui/settings-template.json"

  link="https://huggingface.co/Imablank/AnythingV3-1/raw/main/GPT4.txt, https://huggingface.co/Imablank/AnythingV3-1/raw/main/Alpha.json, https://huggingface.co/Imablank/AnythingV3-1/resolve/main/Alpha.png, https://huggingface.co/Imablank/AnythingV3-1/raw/main/settings.json"
  path="/content/text-generation-webui/presets/, /content/characters/,  /content/characters/, /content/text-generation-webui/"
  nan3="GPT4.txt, Alpha.json, Alpha.png, settings.json"; link=link.split(', '); path=path.split(', '); nan3=nan3.split(', ')
  for link, path, nan3 in zip(link, path, nan3):
    universal_download(link, path, nan3)

  %cd /content/text-generation-webui
  !pip install -r requirements.txt
  !pip install -r extensions/google_translate/requirements.txt
  !pip install -r extensions/api/requirements.txt
  
  if "Wizard-Vicuna-13B-Uncensored-GPTQ" == model or "4bit" in model:
    !mkdir /content/text-generation-webui/repositories
    %cd repositories
    !git clone -b v1.0 https://github.com/camenduru/GPTQ-for-LLaMa.git
    %cd GPTQ-for-LLaMa
    !python setup_cuda.py install

if not os.path.exists(main_dir) or "GPT4-x-alpaca/ROLEPLAY" == model and not os.path.exists("/content/text-generation-webui/repositories"):    
  !cd /content \
    && git clone https://github.com/oobabooga/text-generation-webui \
    && git clone https://github.com/theubie/complex_memory {main_dir}/extensions/complex_memory     
  
  if old_ooba:
    %cd {main_dir}
    !git checkout f052ab9c8fed3dedc446c3847f10ab22e42bfb37

  %cd /content
  clear_output()

  if not Debug:
    display(HTML("<img src='https://huggingface.co/Imablank/AnythingV3-1/resolve/main/BoochiLoading.gif' width='280px'/>"))
    print("\033[92m[Installing oobabooga]")
    with capture.capture_output() as cap:
      install_ooba(); edgegpt()
      if GetAPI: install_silly()
  else:
    install_ooba(); edgegpt()
    if GetAPI: install_silly()
  
else: clear_output(); print("\033[92m\n[ ALREADY INSTALLED -- SKIPPING INSTALL.. ]")

!rm -rf "/content/text-generation-webui/models/config.yaml"

#The Big boy's club
def silly_tavern():
  p = subprocess.Popen(["lt", "--port", "5001"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  Link=(p.stdout.readline().decode().strip())
  # Get external IP address
  external_ip = requests.get('https://api.ipify.org').text
  external_ip=f"\n###Copy Google Colab's Endpoint IP###\nEndpoint IP: \033[4;92m{external_ip}\033[0m"
  print(external_ip, f"\n###SillyTavern LINK###\n{Link}", sep="\n")
  !node server.js

def run_ooba(): 
  cmd = f"python server.py --verbose --model {b} --settings {main_dir}cache/{_username_} {' '.join(params)}"
  print(cmd)
  !$cmd
  if os.path.exists(base_folder):
    username_delete = os.listdir("/content/drive/MyDrive/oobabooga-data/User")
    try: username_delete.remove("pfp_me.png")
    except: pass
    for username_delete in username_delete:
      !rm -rf {base_folder}/User/$username_delete
    if os.path.exists("/content/text-generation-webui/cache/pfp_me.png"):
      ot=" /content/text-generation-webui/cache/pfp_me.png "
    !cp -r /content/text-generation-webui/cache/{_username_}{ot}/content/drive/MyDrive/oobabooga-data/User    


def overwrite_profile():
  global UserName, _username_
  if not username_found == None and not "" == UserName:
    del_userName= '[{'+username_found+"}]-settings-colab.json"
    !rm -rf {cache}/{del_userName}   
  elif not username_found == None and "" == UserName: _username_exists = "[{"+username_found+"}]-settings-colab.json"; UserName = username_found
  elif username_found == None and "" == UserName: UserName="Anon-san"
  %cd {main_dir}
  j = json.loads(open('settings.json', 'r').read())
  j["google_translate-language string"] = language_codes[chat_language]
  j["name1"] = UserName;
  if " " in UserName: UserName=UserName.replace(" ", "_")
  _username_= '[{'+UserName+"}]-settings-colab.json"
  with open(_username_, 'w') as f:
    f.write(json.dumps(j, indent=4))
  !rm -rf {main_dir}cache/{_username_exists}
  !mv /content/text-generation-webui/{_username_} /content/text-generation-webui/cache
  clear_output()

def find_name():
  if os.path.exists(base_folder+"/User"):
    try:
      user_profile = os.listdir(base_folder+"/User")
      for count1 in user_profile:
        !cp -r /content/drive/MyDrive/oobabooga-data/User/$count1 /content/text-generation-webui/cache
    except: pass
  try:
    username_find_dir=os.listdir("/content/text-generation-webui/cache/");
    username_located=", ".join(username_find_dir);
    username_x1=username_located.index("[{")+2;
    username_x2=username_located.index("}]");
    username_found=username_located[username_x1:username_x2]
  except: username_found = None
  finally: return username_found

def download_model(type):
  tmp_repo=f"/content/.{huggingface_repo}"
  !git lfs install --skip-smudge && export GIT_LFS_SKIP_SMUDGE=1 && git clone https://huggingface.co/{huggingface_org}/{huggingface_repo} /content/.{huggingface_repo} --branch {huggingface_branch}
  !rm -rf {tmp_repo}/PygmalionCoT-7b-ggml-model-f16.bin {tmp_repo}/PygmalionCoT-7b-ggml-q4_2.bin {tmp_repo}/PygmalionCoT-7b-ggml-q5_1.bin {tmp_repo}/PygmalionCoT-7b-ggml-q8_0.bin {tmp_repo}/PygmalionCoT-7b-4bit-128g.safetensors {tmp_repo}/pytorch_model.bin.index.json {tmp_repo}/training_args.bin {tmp_repo}/trainer_state.json {tmp_repo}/all_results.json {tmp_repo}/eval_results.json {tmp_repo}/.gitattributes {tmp_repo}/train_results.json
  repo = os.listdir(tmp_repo); clear_output()
  for download in repo:    
    link_path = f"https://huggingface.co/{huggingface_org}/{huggingface_repo}/"
    if ".json" in download or ".txt" in download:
      link_path += f"raw/{huggingface_branch}/{download}"
    else: link_path += f"resolve/{huggingface_branch}/{download}"
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link_path} -d /content/text-generation-webui/models/{type} -o {download}

def upload_profile():  
  #Path reader & png format converter
  pfp = os.listdir("/.pfp/");
  for o in pfp: pfp=o.replace(" ", "_"); pfp="/.pfp/"+o
  if os.path.exists(base_folder):
    if ".png" in o:
      a = os.rename(pfp, "pfp_me.png")
      !mv /.pfp/pfp_me.png /content/drive/MyDrive/oobabooga-data/User
    else:
      Ima1 = Image.open(r""+pfp)
      Ima1.save(r"/content/drive/MyDrive/oobabooga-data/User/pfp_me.png") #cache
  else:
    if ".png" in o:
      a = os.rename(pfp, "pfp_me.png")
      !mv /.pfp/pfp_me.png /content/text-generation-webui/cache
    else:
      Ima1 = Image.open(r""+pfp)
      Ima1.save(r"/content/text-generation-webui/cache/pfp_me.png")

#PART 2 DOWNLOAD AND START
clear_output()
!rm -rf /.pfp/ \
  && mkdir /.pfp/
username_found = find_name()
try:
  if Upload_ProfilePic:
    %cd /.pfp/
    print("\033[92m[Upload Your Profile Picture]\n");files.upload();
    upload_profile()
except: pass

# DOWNLOAD
%cd {main_dir}
if "Pygmalion-350m(CPU)" in model:
  huggingface_org="alpindale";huggingface_repo="pygm-350m-experimental";huggingface_branch="main"
elif "PygmalionCoT-7b" in model:
  huggingface_org="notstoic";huggingface_repo="PygmalionCoT-7b";huggingface_branch="main"
elif "Wizard-Vicuna-13B-Uncensored-GPTQ" in model:
  huggingface_org="TheBloke";huggingface_repo="Wizard-Vicuna-13B-Uncensored-GPTQ";huggingface_branch= "main"
elif "Pygmalion_6B_main_Sharded" in model:
  huggingface_org="waifu-workshop";huggingface_repo="pygmalion-6b";huggingface_branch="sharded"
elif "Pygmalion_6B_original_Sharded" in model:
  huggingface_org="waifu-workshop";huggingface_repo="pygmalion-6b";huggingface_branch="original-sharded"
elif "Pygmalion_6B_dev_Sharded" in model:
  huggingface_org="waifu-workshop";huggingface_repo="pygmalion-6b";huggingface_branch="dev-sharded"
elif "Pygmalion-7B" in model:
  huggingface_org="TehVenom";huggingface_repo="Pygmalion-7b-Merged-Safetensors";huggingface_branch="main"
elif "Metharme-7B" in model:
  huggingface_org="Imablank";huggingface_repo="Metharme-7B-MERGED_WEIGHTS";huggingface_branch="main"
elif "Pygmalion-13B-4bit" in model:
  huggingface_org="notstoic";huggingface_repo="pygmalion-13b-4bit-128g";huggingface_branch="main"
elif "Metharme-13B-4bit" in model:
  huggingface_org="TehVenom";huggingface_repo="Metharme-13b-Merged";huggingface_branch="main"


if not "main" == huggingface_branch: b = huggingface_repo+'_'+huggingface_branch
else: b = huggingface_repo

if not os.path.exists("/content/models/"+huggingface_repo): download_model(b); clear_output(); print("\033[96m\n["+model+" Installed]\n")

clear_output()

language_codes = {'Afrikaans': 'af', 'Albanian': 'sq', 'Amharic': 'am', 'Arabic': 'ar', 'Armenian': 'hy', 'Azerbaijani': 'az', 'Basque': 'eu', 'Belarusian': 'be', 'Bengali': 'bn', 'Bosnian': 'bs', 'Bulgarian': 'bg', 'Catalan': 'ca', 'Cebuano': 'ceb', 'Chinese (Simplified)': 'zh-CN', 'Chinese (Traditional)': 'zh-TW', 'Corsican': 'co', 'Croatian': 'hr', 'Czech': 'cs', 'Danish': 'da', 'Dutch': 'nl', 'English': 'en', 'Esperanto': 'eo', 'Estonian': 'et', 'Finnish': 'fi', 'French': 'fr', 'Frisian': 'fy', 'Galician': 'gl', 'Georgian': 'ka', 'German': 'de', 'Greek': 'el', 'Gujarati': 'gu', 'Haitian Creole': 'ht', 'Hausa': 'ha', 'Hawaiian': 'haw', 'Hebrew': 'iw', 'Hindi': 'hi', 'Hmong': 'hmn', 'Hungarian': 'hu', 'Icelandic': 'is', 'Igbo': 'ig', 'Indonesian': 'id', 'Irish': 'ga', 'Italian': 'it', 'Japanese': 'ja', 'Javanese': 'jw', 'Kannada': 'kn', 'Kazakh': 'kk', 'Khmer': 'km', 'Korean': 'ko', 'Kurdish': 'ku', 'Kyrgyz': 'ky', 'Lao': 'lo', 'Latin': 'la', 'Latvian': 'lv', 'Lithuanian': 'lt', 'Luxembourgish': 'lb', 'Macedonian': 'mk', 'Malagasy': 'mg', 'Malay': 'ms', 'Malayalam': 'ml', 'Maltese': 'mt', 'Maori': 'mi', 'Marathi': 'mr', 'Mongolian': 'mn', 'Myanmar (Burmese)': 'my', 'Nepali': 'ne', 'Norwegian': 'no', 'Nyanja (Chichewa)': 'ny', 'Pashto': 'ps', 'Persian': 'fa', 'Polish': 'pl', 'Portuguese (Portugal, Brazil)': 'pt', 'Punjabi': 'pa', 'Romanian': 'ro', 'Russian': 'ru', 'Samoan': 'sm', 'Scots Gaelic': 'gd', 'Serbian': 'sr', 'Sesotho': 'st', 'Shona': 'sn', 'Sindhi': 'sd', 'Sinhala (Sinhalese)': 'si', 'Slovak': 'sk', 'Slovenian': 'sl', 'Somali': 'so', 'Spanish': 'es', 'Sundanese': 'su', 'Swahili': 'sw', 'Swedish': 'sv', 'Tagalog (Filipino)': 'tl', 'Tajik': 'tg', 'Tamil': 'ta', 'Telugu': 'te', 'Thai': 'th', 'Turkish': 'tr', 'Ukrainian': 'uk', 'Urdu': 'ur', 'Uzbek': 'uz', 'Vietnamese': 'vi', 'Welsh': 'cy', 'Xhosa': 'xh', 'Yiddish': 'yi', 'Yoruba': 'yo', 'Zulu': 'zu'}
overwrite_profile()


print("\033[1m "+"\033[92m ")
params = set()
if "Wizard-Vicuna-13B-Uncensored-GPTQ" in model or "4bit" in model: params.add('--wbits 4 --groupsize 128 --model_type Llama') 

if load_in_8bit: params.add('--load-in-8bit')
params.add('--share --chat') if not GetAPI else params.add('--public-api --chat')
if not text_streaming or activate_google_translate: params.add('--no-stream')

active_extensions = []
if activate_sending_pictures: active_extensions.append('send_pictures')
if activate_character_bias: active_extensions.append('character_bias')
if kobold_memory: active_extensions.append('complex_memory')
if activate_google_translate: active_extensions.append('google_translate')
if GetAPI: active_extensions.append('api')
if GrantBot_Access_to_Bing: active_extensions.append('EdgeGPT')

active_extensions.append('gallery')

if len(active_extensions) > 0: params.add(f'--extensions {" ".join(active_extensions)}')

# Starting the web UI
if RunWebUI:
  run_ooba()
if not Debug: clear_output(); display(HTML("<img src='https://huggingface.co/Imablank/AnythingV3-1/resolve/main/bocchi-the-rock-bocchi.gif' width='300px'/>"))

print("\033[1m "+"\033[92m ")
print("\n[⚠️TEXT-GEN-WEBUI SERVICE TERMINATED⚠️]")

In [ ]:
#@title ##**🚀Start TextGenWebUI🪶**
from IPython.display import clear_output, display, HTML
from IPython.utils import capture
from google.colab import files
from PIL import Image
import os, sys, base64, subprocess, json, shutil, requests, time, pathlib

#PARAMS
#@markdown ####**📖Setup Your UserProfile**

UserName = "" #@param{type:'string'}
Upload_ProfilePic = False #@param{type:'boolean'}
save_logs_to_google_drive = False #@param {type:"boolean"}

#@markdown ######<font color=gray>*Save your data such as your name, logs, characters, etc.. in google drive*
#@markdown ---
#@markdown ####**⚙️Configure and Run🚀**
model = "Pygmalion-350m(CPU)" #@param ["Pygmalion-13B-4bit", "Metharme-13B-4bit", "PygmalionCoT-7b", "Pygmalion-7B", "Metharme-7B", "Wizard-Vicuna-13B-Uncensored-GPTQ", "Pygmalion-350m(CPU)", "Pygmalion_6B_main_Sharded", "Pygmalion_6B_original_Sharded", "Pygmalion_6B_dev_Sharded"]
text_streaming = False #@param {type:"boolean"}
activate_sending_pictures = False #@param {type:"boolean"}
activate_character_bias = False #@param {type:"boolean"}

GrantBot_Access_to_Bing = False #param {type:"boolean"}

chat_language = "English" #@param ['Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Azerbaijani', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Bulgarian', 'Catalan', 'Cebuano', 'Chinese (Simplified)', 'Chinese (Traditional)', 'Corsican', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Esperanto', 'Estonian', 'Finnish', 'French', 'Frisian', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hmong', 'Hungarian', 'Icelandic', 'Igbo', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Kurdish', 'Kyrgyz', 'Lao', 'Latin', 'Latvian', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Mongolian', 'Myanmar (Burmese)', 'Nepali', 'Norwegian', 'Nyanja (Chichewa)', 'Pashto', 'Persian', 'Polish', 'Portuguese (Portugal, Brazil)', 'Punjabi', 'Romanian', 'Russian', 'Samoan', 'Scots Gaelic', 'Serbian', 'Sesotho', 'Shona', 'Sindhi', 'Sinhala (Sinhalese)', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog (Filipino)', 'Tajik', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Welsh', 'Xhosa', 'Yiddish', 'Yoruba', 'Zulu']
activate_google_translate = (chat_language != "English")
RunWebUI = True #@param {type:"boolean"}
GetAPI = False #@param {type:"boolean"}
#@markdown > <font color=#eab676>Run SillyTavern instead of gradio or Get the oobabooga public API .
Debug = True #@param{type:'boolean'}
#@markdown > <font color=gray> *no bocchi? :(*

#@markdown &nbsp; &nbsp;<img src="https://huggingface.co/Imablank/AnythingV3-1/resolve/main/ooba/bocchi-the-rock-crying.gif?size=90" width="90"/> 

dig="""
ot=" "
load_in_8bit = True
if "Wizard-Vicuna-13B-Uncensored-GPTQ" in model or "4bit" in model: load_in_8bit = False
old_ooba = True
kobold_memory = True

#Main Directories:
base_folder = "/content/drive/MyDrive/oobabooga-data"
main_dir = "/content/text-generation-webui/"
cache = f"{main_dir}cache"

#PART 1 INSTALL TextGeN
if save_logs_to_google_drive:
  from google.colab import drive
  drive.mount('/content/drive')

def install_silly():
  ForceInitSteps = []
  class IncrementialInstall:
    def __init__(self, root = "/", tasks = [], force = []):
        self.tasks = tasks
        self.path = os.path.join(root, ".ii")
        self.completed = list(filter(lambda x: not x in force, self.__completed()))

    def __completed(self):
        try:
            with open(self.path) as f:
                return json.load(f)
        except:
            return []

    def addTask(self, name, func):
        self.tasks.append({"name": name, "func": func})

    def run(self):
        todo = list(filter(lambda x: not x["name"] in self.completed, self.tasks))
        try:
            for task in todo:
                task["func"]()
                self.completed.append(task["name"])
        finally:
            with open(self.path, "w") as f:
                json.dump(self.completed, f)

  def create_paths(paths):
    for directory in paths:
        if not os.path.exists(directory):
            os.makedirs(directory)

  def link(srcDir, destDir, files):
    '''
        Link source to dest copying dest to source if not present first
    '''
    for file in files:
        source = os.path.join(srcDir, file)
        dest = os.path.join(destDir, file)
        if not os.path.exists(source):
            !cp -r "$dest" "$source"
        !rm -rf "$dest"
        !ln -fs "$source" "$dest"
  from google.colab import drive
  if not save_logs_to_google_drive:
    create_paths([
            "/content/SillyTavern-Data"
    ])
  ii = IncrementialInstall(force=ForceInitSteps)
# ---
# SillyTavern py modules
  def cloneTavern():
    %cd /
    !git clone https://github.com/Cohee1207/SillyTavern
    !wget https://huggingface.co/Imablank/AnythingV3-1/resolve/main/ooba/globals.py && wget https://huggingface.co/Imablank/AnythingV3-1/resolve/main/ooba/extras_server.py 
  ii.addTask("Clone SillyTavern", cloneTavern)
  ii.run()
  kargs = ["/content/ckds"]
  kargs += ["--localtunnel", "yes"]

# ---
# nodejs
  %cd /
  def installNode():
    !npm install -g n
    !n 19
    !node --version
  installNode()
# ---
# TavernAI extras
  import globals
  globals.params = []
  globals.params.append('--cpu')
  %cd /SillyTavern
  if save_logs_to_google_drive:
    %env googledrive=2
    def setupTavernPaths():
        %cd /SillyTavern
        tdrive = "/content/drive/MyDrive/SillyTavern"
        create_paths([
                tdrive,
                os.path.join("public", "groups"),
                os.path.join("public", "group chats")
        ])
        link(tdrive, "public", [
                "settings.json",
                "backgrounds",
                "characters",
                "chats",
                "User Avatars",
                "worlds",
                "group chats",
                "groups",
        ])
    ii.addTask("Setup Tavern Paths", setupTavernPaths)
  def installTavernDependencies():
    %cd /SillyTavern
    !npm install
    !npm install -g localtunnel
  ii.addTask("Install Tavern Dependencies", installTavernDependencies)
  ii.run()

def universal_download(link1, path1, nan3):
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link1} -d {path1} -o {nan3}

def edgegpt():
  import zipfile
  %cd /content/
  !wget https://huggingface.co/Imablank/AnythingV3-1/resolve/main/ooba/EdgeGPT_working.zip
  with zipfile.ZipFile('/content/EdgeGPT_working.zip', 'r') as zip:
    zip.extractall("/content/text-generation-webui/extensions")
  !pip install -r {main_dir}extensions/EdgeGPT/requirements.txt && rm -rf /content/EdgeGPT_working.zip
  %cd {main_dir}

def install_ooba():
  !apt-get -y install -qq aria2
     
  !mkdir {main_dir}cache
  if save_logs_to_google_drive:
    if not os.path.exists(f"{base_folder}"):
      os.mkdir(f"{base_folder}")
    if not os.path.exists(f"{base_folder}/logs"):
      os.mkdir(f"{base_folder}/logs")
    if not os.path.exists(f"{base_folder}/User"):
      os.mkdir(f"{base_folder}/User")
    if not os.path.exists(f"{base_folder}/softprompts"):
      os.mkdir(f"{base_folder}/softprompts")
    if not os.path.exists(f"{base_folder}/characters"):
      shutil.move("text-generation-webui/characters", f"{base_folder}/characters")
    else:
      !rm -r "text-generation-webui/characters"
    
    !rm -r "text-generation-webui/softprompts"
    !ln -s "$base_folder/logs" "text-generation-webui/"
    !ln -s "$base_folder/softprompts" "text-generation-webui/softprompts"
    !ln -s "$base_folder/characters" "text-generation-webui/characters"
    !ln -s "$base_folder/User" .
  else:
    !mkdir text-generation-webui/logs
    
  !ln -s text-generation-webui/logs .
  !ln -s text-generation-webui/characters .
  !ln -s text-generation-webui/models .
    
  %rm -r sample_data
  %cd {main_dir}
  !wget https://oobabooga.github.io/settings-colab.json -O settings-colab-template.json

  !rm -rf "/content/text-generation-webui/presets/Default.txt" "/content/text-generation-webui/settings-colab-template.json" "/content/text-generation-webui/settings-template.json"

  link="https://huggingface.co/Imablank/AnythingV3-1/raw/main/GPT4.txt, https://huggingface.co/Imablank/AnythingV3-1/raw/main/Alpha.json, https://huggingface.co/Imablank/AnythingV3-1/resolve/main/Alpha.png, https://huggingface.co/Imablank/AnythingV3-1/raw/main/settings.json"
  path="/content/text-generation-webui/presets/, /content/characters/,  /content/characters/, /content/text-generation-webui/"
  nan3="GPT4.txt, Alpha.json, Alpha.png, settings.json"; link=link.split(', '); path=path.split(', '); nan3=nan3.split(', ')
  for link, path, nan3 in zip(link, path, nan3):
    universal_download(link, path, nan3)

  %cd /content/text-generation-webui
  !pip install -r requirements.txt
  !pip install -r extensions/google_translate/requirements.txt
  !pip install -r extensions/api/requirements.txt
  
  if "Wizard-Vicuna-13B-Uncensored-GPTQ" == model or "4bit" in model:
    !mkdir /content/text-generation-webui/repositories
    %cd repositories
    !git clone -b v1.0 https://github.com/camenduru/GPTQ-for-LLaMa.git
    %cd GPTQ-for-LLaMa
    !python setup_cuda.py install

if not os.path.exists(main_dir) or "GPT4-x-alpaca/ROLEPLAY" == model and not os.path.exists("/content/text-generation-webui/repositories"):    
  !cd /content \
    && git clone https://github.com/oobabooga/text-generation-webui \
    && git clone https://github.com/theubie/complex_memory {main_dir}/extensions/complex_memory     
  
  if old_ooba:
    %cd {main_dir}
    !git checkout f052ab9c8fed3dedc446c3847f10ab22e42bfb37

  %cd /content
  clear_output()

  if not Debug:
    display(HTML("<img src='https://huggingface.co/Imablank/AnythingV3-1/resolve/main/BoochiLoading.gif' width='280px'/>"))
    print("\033[92m[Installing oobabooga]")
    with capture.capture_output() as cap:
      install_ooba(); edgegpt()
      if GetAPI: install_silly()
  else:
    install_ooba(); edgegpt()
    if GetAPI: install_silly()
  
else: clear_output(); print("\033[92m\n[ ALREADY INSTALLED -- SKIPPING INSTALL.. ]")

!rm -rf "/content/text-generation-webui/models/config.yaml"

#The Big boy's club
def silly_tavern():
  p = subprocess.Popen(["lt", "--port", "5001"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  Link=(p.stdout.readline().decode().strip())
  # Get external IP address
  external_ip = requests.get('https://api.ipify.org').text
  external_ip=f"\n###Copy Google Colab's Endpoint IP###\nEndpoint IP: \033[4;92m{external_ip}\033[0m"
  print(external_ip, f"\n###SillyTavern LINK###\n{Link}", sep="\n")
  !node server.js

def run_ooba(): 
  cmd = f"python server.py --verbose --model {b} --settings {main_dir}cache/{_username_} {' '.join(params)}"
  print(cmd)
  !$cmd
  if os.path.exists(base_folder):
    username_delete = os.listdir("/content/drive/MyDrive/oobabooga-data/User")
    try: username_delete.remove("pfp_me.png")
    except: pass
    for username_delete in username_delete:
      !rm -rf {base_folder}/User/$username_delete
    if os.path.exists("/content/text-generation-webui/cache/pfp_me.png"):
      ot=" /content/text-generation-webui/cache/pfp_me.png "
    !cp -r /content/text-generation-webui/cache/{_username_}{ot}/content/drive/MyDrive/oobabooga-data/User    


def overwrite_profile():
  global UserName, _username_
  if not username_found == None and not "" == UserName:
    del_userName= '[{'+username_found+"}]-settings-colab.json"
    !rm -rf {cache}/{del_userName}   
  elif not username_found == None and "" == UserName: _username_exists = "[{"+username_found+"}]-settings-colab.json"; UserName = username_found
  elif username_found == None and "" == UserName: UserName="Anon-san"
  %cd {main_dir}
  j = json.loads(open('settings.json', 'r').read())
  j["google_translate-language string"] = language_codes[chat_language]
  j["name1"] = UserName;
  if " " in UserName: UserName=UserName.replace(" ", "_")
  _username_= '[{'+UserName+"}]-settings-colab.json"
  with open(_username_, 'w') as f:
    f.write(json.dumps(j, indent=4))
  !rm -rf {main_dir}cache/{_username_exists}
  !mv /content/text-generation-webui/{_username_} /content/text-generation-webui/cache
  clear_output()

def find_name():
  if os.path.exists(base_folder+"/User"):
    try:
      user_profile = os.listdir(base_folder+"/User")
      for count1 in user_profile:
        !cp -r /content/drive/MyDrive/oobabooga-data/User/$count1 /content/text-generation-webui/cache
    except: pass
  try:
    username_find_dir=os.listdir("/content/text-generation-webui/cache/");
    username_located=", ".join(username_find_dir);
    username_x1=username_located.index("[{")+2;
    username_x2=username_located.index("}]");
    username_found=username_located[username_x1:username_x2]
  except: username_found = None
  finally: return username_found

def download_model(type):
  tmp_repo=f"/content/.{huggingface_repo}"
  !git lfs install --skip-smudge && export GIT_LFS_SKIP_SMUDGE=1 && git clone https://huggingface.co/{huggingface_org}/{huggingface_repo} /content/.{huggingface_repo} --branch {huggingface_branch}
  !rm -rf {tmp_repo}/PygmalionCoT-7b-ggml-model-f16.bin {tmp_repo}/PygmalionCoT-7b-ggml-q4_2.bin {tmp_repo}/PygmalionCoT-7b-ggml-q5_1.bin {tmp_repo}/PygmalionCoT-7b-ggml-q8_0.bin {tmp_repo}/PygmalionCoT-7b-4bit-128g.safetensors {tmp_repo}/pytorch_model.bin.index.json {tmp_repo}/training_args.bin {tmp_repo}/trainer_state.json {tmp_repo}/all_results.json {tmp_repo}/eval_results.json {tmp_repo}/.gitattributes {tmp_repo}/train_results.json
  repo = os.listdir(tmp_repo); clear_output()
  for download in repo:    
    link_path = f"https://huggingface.co/{huggingface_org}/{huggingface_repo}/"
    if ".json" in download or ".txt" in download:
      link_path += f"raw/{huggingface_branch}/{download}"
    else: link_path += f"resolve/{huggingface_branch}/{download}"
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link_path} -d /content/text-generation-webui/models/{type} -o {download}

def upload_profile():  
  #Path reader & png format converter
  pfp = os.listdir("/.pfp/");
  for o in pfp: pfp=o.replace(" ", "_"); pfp="/.pfp/"+o
  if os.path.exists(base_folder):
    if ".png" in o:
      a = os.rename(pfp, "pfp_me.png")
      !mv /.pfp/pfp_me.png /content/drive/MyDrive/oobabooga-data/User
    else:
      Ima1 = Image.open(r""+pfp)
      Ima1.save(r"/content/drive/MyDrive/oobabooga-data/User/pfp_me.png") #cache
  else:
    if ".png" in o:
      a = os.rename(pfp, "pfp_me.png")
      !mv /.pfp/pfp_me.png /content/text-generation-webui/cache
    else:
      Ima1 = Image.open(r""+pfp)
      Ima1.save(r"/content/text-generation-webui/cache/pfp_me.png")

#PART 2 DOWNLOAD AND START
clear_output()
!rm -rf /.pfp/ \
  && mkdir /.pfp/
username_found = find_name()
try:
  if Upload_ProfilePic:
    %cd /.pfp/
    print("\033[92m[Upload Your Profile Picture]\n");files.upload();
    upload_profile()
except: pass

# DOWNLOAD
%cd {main_dir}
if "Pygmalion-350m(CPU)" in model:
  huggingface_org="alpindale";huggingface_repo="pygm-350m-experimental";huggingface_branch="main"
elif "PygmalionCoT-7b" in model:
  huggingface_org="notstoic";huggingface_repo="PygmalionCoT-7b";huggingface_branch="main"
elif "Wizard-Vicuna-13B-Uncensored-GPTQ" in model:
  huggingface_org="TheBloke";huggingface_repo="Wizard-Vicuna-13B-Uncensored-GPTQ";huggingface_branch= "main"
elif "Pygmalion_6B_main_Sharded" in model:
  huggingface_org="waifu-workshop";huggingface_repo="pygmalion-6b";huggingface_branch="sharded"
elif "Pygmalion_6B_original_Sharded" in model:
  huggingface_org="waifu-workshop";huggingface_repo="pygmalion-6b";huggingface_branch="original-sharded"
elif "Pygmalion_6B_dev_Sharded" in model:
  huggingface_org="waifu-workshop";huggingface_repo="pygmalion-6b";huggingface_branch="dev-sharded"
elif "Pygmalion-7B" in model:
  huggingface_org="TehVenom";huggingface_repo="Pygmalion-7b-Merged-Safetensors";huggingface_branch="main"
elif "Metharme-7B" in model:
  huggingface_org="Imablank";huggingface_repo="Metharme-7B-MERGED_WEIGHTS";huggingface_branch="main"
elif "Pygmalion-13B-4bit" in model:
  huggingface_org="notstoic";huggingface_repo="pygmalion-13b-4bit-128g";huggingface_branch="main"
elif "Metharme-13B-4bit" in model:
  huggingface_org="TehVenom";huggingface_repo="Metharme-13b-Merged";huggingface_branch="main"


if not "main" == huggingface_branch: b = huggingface_repo+'_'+huggingface_branch
else: b = huggingface_repo

if not os.path.exists("/content/models/"+huggingface_repo): download_model(b); clear_output(); print("\033[96m\n["+model+" Installed]\n")

clear_output()

language_codes = {'Afrikaans': 'af', 'Albanian': 'sq', 'Amharic': 'am', 'Arabic': 'ar', 'Armenian': 'hy', 'Azerbaijani': 'az', 'Basque': 'eu', 'Belarusian': 'be', 'Bengali': 'bn', 'Bosnian': 'bs', 'Bulgarian': 'bg', 'Catalan': 'ca', 'Cebuano': 'ceb', 'Chinese (Simplified)': 'zh-CN', 'Chinese (Traditional)': 'zh-TW', 'Corsican': 'co', 'Croatian': 'hr', 'Czech': 'cs', 'Danish': 'da', 'Dutch': 'nl', 'English': 'en', 'Esperanto': 'eo', 'Estonian': 'et', 'Finnish': 'fi', 'French': 'fr', 'Frisian': 'fy', 'Galician': 'gl', 'Georgian': 'ka', 'German': 'de', 'Greek': 'el', 'Gujarati': 'gu', 'Haitian Creole': 'ht', 'Hausa': 'ha', 'Hawaiian': 'haw', 'Hebrew': 'iw', 'Hindi': 'hi', 'Hmong': 'hmn', 'Hungarian': 'hu', 'Icelandic': 'is', 'Igbo': 'ig', 'Indonesian': 'id', 'Irish': 'ga', 'Italian': 'it', 'Japanese': 'ja', 'Javanese': 'jw', 'Kannada': 'kn', 'Kazakh': 'kk', 'Khmer': 'km', 'Korean': 'ko', 'Kurdish': 'ku', 'Kyrgyz': 'ky', 'Lao': 'lo', 'Latin': 'la', 'Latvian': 'lv', 'Lithuanian': 'lt', 'Luxembourgish': 'lb', 'Macedonian': 'mk', 'Malagasy': 'mg', 'Malay': 'ms', 'Malayalam': 'ml', 'Maltese': 'mt', 'Maori': 'mi', 'Marathi': 'mr', 'Mongolian': 'mn', 'Myanmar (Burmese)': 'my', 'Nepali': 'ne', 'Norwegian': 'no', 'Nyanja (Chichewa)': 'ny', 'Pashto': 'ps', 'Persian': 'fa', 'Polish': 'pl', 'Portuguese (Portugal, Brazil)': 'pt', 'Punjabi': 'pa', 'Romanian': 'ro', 'Russian': 'ru', 'Samoan': 'sm', 'Scots Gaelic': 'gd', 'Serbian': 'sr', 'Sesotho': 'st', 'Shona': 'sn', 'Sindhi': 'sd', 'Sinhala (Sinhalese)': 'si', 'Slovak': 'sk', 'Slovenian': 'sl', 'Somali': 'so', 'Spanish': 'es', 'Sundanese': 'su', 'Swahili': 'sw', 'Swedish': 'sv', 'Tagalog (Filipino)': 'tl', 'Tajik': 'tg', 'Tamil': 'ta', 'Telugu': 'te', 'Thai': 'th', 'Turkish': 'tr', 'Ukrainian': 'uk', 'Urdu': 'ur', 'Uzbek': 'uz', 'Vietnamese': 'vi', 'Welsh': 'cy', 'Xhosa': 'xh', 'Yiddish': 'yi', 'Yoruba': 'yo', 'Zulu': 'zu'}
overwrite_profile()


print("\033[1m "+"\033[92m ")
params = set()
if "Wizard-Vicuna-13B-Uncensored-GPTQ" in model or "4bit" in model: params.add('--wbits 4 --groupsize 128 --model_type Llama') 

if load_in_8bit: params.add('--load-in-8bit')
params.add('--share --chat') if not GetAPI else params.add('--public-api --chat')
if not text_streaming or activate_google_translate: params.add('--no-stream')

active_extensions = []
if activate_sending_pictures: active_extensions.append('send_pictures')
if activate_character_bias: active_extensions.append('character_bias')
if kobold_memory: active_extensions.append('complex_memory')
if activate_google_translate: active_extensions.append('google_translate')
if GetAPI: active_extensions.append('api')
if GrantBot_Access_to_Bing: active_extensions.append('EdgeGPT')

active_extensions.append('gallery')

if len(active_extensions) > 0: params.add(f'--extensions {" ".join(active_extensions)}')

# Starting the web UI
if RunWebUI:
  run_ooba()
if not Debug: clear_output(); display(HTML("<img src='https://huggingface.co/Imablank/AnythingV3-1/resolve/main/bocchi-the-rock-bocchi.gif' width='300px'/>"))

print("\033[1m "+"\033[92m ")
print("\n[⚠️TEXT-GEN-WEBUI SERVICE TERMINATED⚠️]")
"""

In [ ]:
#@title ##**🚀Start TextGenWebUI**
#GOOGLE COLAB ONLY
import sys
sys.path.append("/")
%cd /
!wget https://huggingface.co/Imablank/AnythingV3-1/resolve/main/ooba/Run_WebUI.py
%cd -

from Run_WebUI import *

from IPython.display import clear_output, display, HTML
from IPython.utils import capture
from google.colab import files
from PIL import Image
import os, sys, base64, subprocess, json, shutil, requests, time, pathlib

#PARAMS
#@markdown ####**📖Setup Your UserProfile**

UserName = "" #@param{type:'string'}
Upload_ProfilePic = False #@param{type:'boolean'}
save_logs_to_google_drive = False #@param {type:"boolean"}

#@markdown ######<font color=gray>*Save your data such as your name, logs, characters, etc.. in google drive*
#@markdown ---
#@markdown ####**⚙️Configure and Run🚀**
model = "Pygmalion-350m(CPU)" #@param ["Pygmalion-13B-4bit", "Metharme-13B-4bit", "PygmalionCoT-7b", "Pygmalion-7B", "Metharme-7B", "Wizard-Vicuna-13B-Uncensored-GPTQ", "Pygmalion-350m(CPU)", "Pygmalion_6B_main_Sharded", "Pygmalion_6B_original_Sharded", "Pygmalion_6B_dev_Sharded"]
text_streaming = False #@param {type:"boolean"}
activate_sending_pictures = False #@param {type:"boolean"}
activate_character_bias = False #@param {type:"boolean"}

GrantBot_Access_to_Bing = False #param {type:"boolean"}

chat_language = "English" #@param ['Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Azerbaijani', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Bulgarian', 'Catalan', 'Cebuano', 'Chinese (Simplified)', 'Chinese (Traditional)', 'Corsican', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Esperanto', 'Estonian', 'Finnish', 'French', 'Frisian', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hmong', 'Hungarian', 'Icelandic', 'Igbo', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Kurdish', 'Kyrgyz', 'Lao', 'Latin', 'Latvian', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Mongolian', 'Myanmar (Burmese)', 'Nepali', 'Norwegian', 'Nyanja (Chichewa)', 'Pashto', 'Persian', 'Polish', 'Portuguese (Portugal, Brazil)', 'Punjabi', 'Romanian', 'Russian', 'Samoan', 'Scots Gaelic', 'Serbian', 'Sesotho', 'Shona', 'Sindhi', 'Sinhala (Sinhalese)', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog (Filipino)', 'Tajik', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Welsh', 'Xhosa', 'Yiddish', 'Yoruba', 'Zulu']
activate_google_translate = (chat_language != "English")
RunWebUI = True #@param {type:"boolean"}
GetAPI = False #@param {type:"boolean"}
#@markdown > <font color=#eab676>Run SillyTavern instead of gradio or Get the oobabooga public API .
Debug = True #@param{type:'boolean'}
#@markdown > <font color=gray> *no bocchi? :(*

#@markdown &nbsp; &nbsp;<img src="https://huggingface.co/Imablank/AnythingV3-1/resolve/main/ooba/bocchi-the-rock-crying.gif?size=90" width="90"/> 

if not Debug:
  display(HTML("<img src='https://huggingface.co/Imablank/AnythingV3-1/resolve/main/BoochiLoading.gif' width='280px'/>"))
  print("\033[92m[Installing oobabooga]")
  with capture.capture_output() as cap:
    install_ooba(); edgegpt()
    if GetAPI: install_silly()
else:
  install_ooba(); edgegpt()
  if GetAPI: install_silly()


# OOBA ONLY
def upload_profile():  
  #Path reader & png format converter
  pfp = os.listdir("/.pfp/");
  for o in pfp: pfp=o.replace(" ", "_"); pfp=f"/.pfp/{o}"
  if os.path.exists(base_folder):
    if ".png" in o:
      a = os.rename(pfp, "pfp_me.png")
      os.system(f"mv /.pfp/pfp_me.png /content/drive/MyDrive/oobabooga-data/User")
    else:
      Ima1 = Image.open(r""+pfp)
      Ima1.save(r"/content/drive/MyDrive/oobabooga-data/User/pfp_me.png") #cache
  else:
    if ".png" in o:
      a = os.rename(pfp, "pfp_me.png")
      os.system(f"mv /.pfp/pfp_me.png /content/text-generation-webui/cache")
    else:
      Ima1 = Image.open(r""+pfp)
      Ima1.save(r"/content/text-generation-webui/cache/pfp_me.png")

clear_output()
os.system(f"rm -rf /.pfp/ && mkdir /.pfp/")

#Upload Profile pic in ooba
try:
  if Upload_ProfilePic:
    %cd /.pfp/
    print("\033[92m[Upload Your Profile Picture]\n");files.upload();
    
    upload_profile()
except: pass

language_codes = {'Afrikaans': 'af', 'Albanian': 'sq', 'Amharic': 'am', 'Arabic': 'ar', 'Armenian': 'hy', 'Azerbaijani': 'az', 'Basque': 'eu', 'Belarusian': 'be', 'Bengali': 'bn', 'Bosnian': 'bs', 'Bulgarian': 'bg', 'Catalan': 'ca', 'Cebuano': 'ceb', 'Chinese (Simplified)': 'zh-CN', 'Chinese (Traditional)': 'zh-TW', 'Corsican': 'co', 'Croatian': 'hr', 'Czech': 'cs', 'Danish': 'da', 'Dutch': 'nl', 'English': 'en', 'Esperanto': 'eo', 'Estonian': 'et', 'Finnish': 'fi', 'French': 'fr', 'Frisian': 'fy', 'Galician': 'gl', 'Georgian': 'ka', 'German': 'de', 'Greek': 'el', 'Gujarati': 'gu', 'Haitian Creole': 'ht', 'Hausa': 'ha', 'Hawaiian': 'haw', 'Hebrew': 'iw', 'Hindi': 'hi', 'Hmong': 'hmn', 'Hungarian': 'hu', 'Icelandic': 'is', 'Igbo': 'ig', 'Indonesian': 'id', 'Irish': 'ga', 'Italian': 'it', 'Japanese': 'ja', 'Javanese': 'jw', 'Kannada': 'kn', 'Kazakh': 'kk', 'Khmer': 'km', 'Korean': 'ko', 'Kurdish': 'ku', 'Kyrgyz': 'ky', 'Lao': 'lo', 'Latin': 'la', 'Latvian': 'lv', 'Lithuanian': 'lt', 'Luxembourgish': 'lb', 'Macedonian': 'mk', 'Malagasy': 'mg', 'Malay': 'ms', 'Malayalam': 'ml', 'Maltese': 'mt', 'Maori': 'mi', 'Marathi': 'mr', 'Mongolian': 'mn', 'Myanmar (Burmese)': 'my', 'Nepali': 'ne', 'Norwegian': 'no', 'Nyanja (Chichewa)': 'ny', 'Pashto': 'ps', 'Persian': 'fa', 'Polish': 'pl', 'Portuguese (Portugal, Brazil)': 'pt', 'Punjabi': 'pa', 'Romanian': 'ro', 'Russian': 'ru', 'Samoan': 'sm', 'Scots Gaelic': 'gd', 'Serbian': 'sr', 'Sesotho': 'st', 'Shona': 'sn', 'Sindhi': 'sd', 'Sinhala (Sinhalese)': 'si', 'Slovak': 'sk', 'Slovenian': 'sl', 'Somali': 'so', 'Spanish': 'es', 'Sundanese': 'su', 'Swahili': 'sw', 'Swedish': 'sv', 'Tagalog (Filipino)': 'tl', 'Tajik': 'tg', 'Tamil': 'ta', 'Telugu': 'te', 'Thai': 'th', 'Turkish': 'tr', 'Ukrainian': 'uk', 'Urdu': 'ur', 'Uzbek': 'uz', 'Vietnamese': 'vi', 'Welsh': 'cy', 'Xhosa': 'xh', 'Yiddish': 'yi', 'Yoruba': 'yo', 'Zulu': 'zu'}


username_found = oobabooga.find_name()
oobabooga.overwrite_profile()

print("\033[1m "+"\033[92m ")

os.chdir(main_dir)
if "Pygmalion-350m(CPU)" in model:
  huggingface_org="alpindale";huggingface_repo="pygm-350m-experimental";huggingface_branch="main"
elif "PygmalionCoT-7b" in model:
  huggingface_org="notstoic";huggingface_repo="PygmalionCoT-7b";huggingface_branch="main"
elif "Wizard-Vicuna-13B-Uncensored-GPTQ" in model:
  huggingface_org="TheBloke";huggingface_repo="Wizard-Vicuna-13B-Uncensored-GPTQ";huggingface_branch= "main"
elif "Pygmalion_6B_main_Sharded" in model:
  huggingface_org="waifu-workshop";huggingface_repo="pygmalion-6b";huggingface_branch="sharded"
elif "Pygmalion_6B_original_Sharded" in model:
  huggingface_org="waifu-workshop";huggingface_repo="pygmalion-6b";huggingface_branch="original-sharded"
elif "Pygmalion_6B_dev_Sharded" in model:
  huggingface_org="waifu-workshop";huggingface_repo="pygmalion-6b";huggingface_branch="dev-sharded"
elif "Pygmalion-7B" in model:
  huggingface_org="TehVenom";huggingface_repo="Pygmalion-7b-Merged-Safetensors";huggingface_branch="main"
elif "Metharme-7B" in model:
  huggingface_org="Imablank";huggingface_repo="Metharme-7B-MERGED_WEIGHTS";huggingface_branch="main"
elif "Pygmalion-13B-4bit" in model:
  huggingface_org="notstoic";huggingface_repo="pygmalion-13b-4bit-128g";huggingface_branch="main"
elif "Metharme-13B-4bit" in model:
  huggingface_org="TehVenom";huggingface_repo="Metharme-13b-Merged";huggingface_branch="main"

if not "main" == huggingface_branch: b = huggingface_repo+'_'+huggingface_branch
else: b = huggingface_repo

if not os.path.exists(f"/content/models/{huggingface_repo}"):
  download_model(b);
  clear_output(); print(f"\033[96m\n[{model} Installed]\n")



clear_output()


# Starting the web UI
if RunWebUI:
  #run_silly_tavern()
  run_ooba()
if not Debug: clear_output(); display(HTML("<img src='https://huggingface.co/Imablank/AnythingV3-1/resolve/main/bocchi-the-rock-bocchi.gif' width='300px'/>"))

print("\033[1m "+"\033[92m ")
print("\n[TEXT-GEN-WEBUI SERVICE TERMINATED]")



In [ ]:
#Receive
"""
var_list=sys.argv[1]

#Main Variables

UserName=var_list[0]
save_logs_to_google_drive=var_list[1]
model=var_list[2]
text_streaming[3]
activate_sending_pictures=var_list[4]
activate_character_bias=var_list[5]
chat_language=var_list[6]
RunWebUI=var_list[7]
GetAPI=var_list[8]
Debug=var_list[9]

activate_google_translate = (chat_language != "English")
"""


"""
ACCESS TOOLS

IMPORTANT CLASS AND FUNCTIONs:

def universal_download(link1, path1, nan3) = DOWNLOAD EVERYTHING
def install_silly() = EVERYTHING SILLY
class oobabooga = EVERYTHING OOBA
class utils = MAIN CLASS TOOLS

"""

#PART 1 INSTALL 

def universal_download(link1, path1, nan3):
  os.system(f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link1} -d {path1} -o {nan3}")

#SillyTavern
def install_silly():
  ForceInitSteps = []
  class IncrementialInstall:
    def __init__(self, root = "/", tasks = [], force = []):
        self.tasks = tasks
        self.path = os.path.join(root, ".ii")
        self.completed = list(filter(lambda x: not x in force, self.__completed()))
    def __completed(self):
        try:
            with open(self.path) as f:
                return json.load(f)
        except:
            return []
    def addTask(self, name, func):
        self.tasks.append({"name": name, "func": func})
    def run(self):
        todo = list(filter(lambda x: not x["name"] in self.completed, self.tasks))
        try:
            for task in todo:
                task["func"]()
                self.completed.append(task["name"])
        finally:
            with open(self.path, "w") as f:
                json.dump(self.completed, f)
  def create_paths(paths):
    for directory in paths:
        if not os.path.exists(directory):
            os.makedirs(directory)
  def link(srcDir, destDir, files):
    '''
        Link source to dest copying dest to source if not present first
    '''
    for file in files:
        source = os.path.join(srcDir, file)
        dest = os.path.join(destDir, file)
        if not os.path.exists(source):
            os.system(f"cp -r {dest} {source}")
        os.system(f"rm -rf {dest}")
        os.system(f"ln -fs {source} {dest}")
  from google.colab import drive
  if not save_logs_to_google_drive:
    create_paths([
            "/content/SillyTavern-Data"
    ])
  ii = IncrementialInstall(force=ForceInitSteps)
# ---
# SillyTavern py modules
  def cloneTavern():
    os.chdir('/')
    os.system(f"git clone https://github.com/Cohee1207/SillyTavern")
    os.system(f"wget https://huggingface.co/Imablank/AnythingV3-1/resolve/main/ooba/globals.py && wget https://huggingface.co/Imablank/AnythingV3-1/resolve/main/ooba/extras_server.py")
  ii.addTask("Clone SillyTavern", cloneTavern)
  ii.run()
  kargs = ["/content/ckds"]
  kargs += ["--localtunnel", "yes"]

# ---
# nodejs
  os.chdir('/')
  def installNode():
    os.system(f"npm install -g n")
    os.system(f"n 19")
    os.system(f"node --version")
  installNode()
# ---
# TavernAI extras
  import globals
  globals.params = []
  globals.params.append('--cpu')
  os.chdir('/SillyTavern') 
  if save_logs_to_google_drive:
    os.system("env googledrive=2") #%env googledrive=2
    def setupTavernPaths():
        os.chdir('/SillyTavern') 
        tdrive = "/content/drive/MyDrive/SillyTavern"
        create_paths([
                tdrive,
                os.path.join("public", "groups"),
                os.path.join("public", "group chats")
        ])
        link(tdrive, "public", [
                "settings.json",
                "backgrounds",
                "characters",
                "chats",
                "User Avatars",
                "worlds",
                "group chats",
                "groups",
        ])
    ii.addTask("Setup Tavern Paths", setupTavernPaths)
  def installTavernDependencies():
    os.chdir('/SillyTavern') 
    os.system("npm install")
    os.system("npm install -g localtunnel")
  ii.addTask("Install Tavern Dependencies", installTavernDependencies)
  ii.run()

#Oobabooga

class oobabooga:
  global quant_, base_folder, main_dir, cache
  #Main Directories:
  base_folder = "/content/drive/MyDrive/oobabooga-data"
  main_dir = "/content/text-generation-webui/"
  cache = f"{main_dir}cache"

  quant_="GPTQ|4bit|128"
  GrantBot_Access_to_Bing = False
  load_in_8bit = True
  if (model in quant_): load_in_8bit = False
  old_ooba = True
  complex_memory = True
  filler=" "

  def edgegpt():
    import zipfile
    os.chdir("/content/")
    os.system(f"wget https://huggingface.co/Imablank/AnythingV3-1/resolve/main/ooba/EdgeGPT_working.zip")
    with zipfile.ZipFile('/content/EdgeGPT_working.zip', 'r') as zip:
        zip.extractall("/content/text-generation-webui/extensions")
    os.system(f"pip install -r {main_dir}extensions/EdgeGPT/requirements.txt && rm -rf /content/EdgeGPT_working.zip")
    os.chdir(main_dir)
    
  def overwrite_profile():
    global UserName, _username_, base_folder, main_dir, cache
    if not username_found == None and not "" == UserName:
        del_userName= '[{'+username_found+"}]-settings-colab.json"
        os.system(f"rm -rf {cache}/{del_userName}")    
    elif not username_found == None and "" == UserName: _username_exists = "[{"+username_found+"}]-settings-colab.json"; UserName = username_found
    elif username_found == None and "" == UserName: UserName="Anon-san"
    os.chdir(main_dir)  
    j = json.loads(open('settings.json', 'r').read())
    j["google_translate-language string"] = language_codes[chat_language]
    j["name1"] = UserName;
    if " " in UserName: UserName=UserName.replace(" ", "_")
    _username_= '[{'+UserName+"}]-settings-colab.json"
    with open(_username_, 'w') as f:
        f.write(json.dumps(j, indent=4))
    os.system(f"rm -rf {main_dir}cache/{_username_exists}")
    os.system(f"mv /content/text-generation-webui/{_username_} /content/text-generation-webui/cache")
    clear_output()

  def find_name():
    global base_folder
    if os.path.exists(base_folder+"/User"):
        try:
            user_profile = os.listdir(base_folder+"/User")
            for count1 in user_profile:
                os.system(f"cp -r /content/drive/MyDrive/oobabooga-data/User/{count1} /content/text-generation-webui/cache")
        except: pass
    try:
        username_find_dir=os.listdir("/content/text-generation-webui/cache/");
        username_located=", ".join(username_find_dir);
        username_x1=username_located.index("[{")+2;
        username_x2=username_located.index("}]");
        username_found=username_located[username_x1:username_x2]
    except: username_found = None
    finally: return username_found

  def install_ooba():
    global quant_, base_folder, main_dir, cache
    os.system(f"apt-get -y install -qq aria2 && mkdir {main_dir}cache")
    if save_logs_to_google_drive:
        if not os.path.exists(f"{base_folder}"):
            os.mkdir(f"{base_folder}")
        if not os.path.exists(f"{base_folder}/logs"):
            os.mkdir(f"{base_folder}/logs")
        if not os.path.exists(f"{base_folder}/User"):
            os.mkdir(f"{base_folder}/User")
        if not os.path.exists(f"{base_folder}/softprompts"):
            os.mkdir(f"{base_folder}/softprompts")
        if not os.path.exists(f"{base_folder}/characters"):
            shutil.move("text-generation-webui/characters", f"{base_folder}/characters")
        else:
            os.system(f"rm -rf {main_dir}characters")
        
        os.system(f"rm -r text-generation-webui/softprompts")
        os.system(f"ln -s $base_folder/logs text-generation-webui/")
        os.system(f"ln -s $base_folder/softprompts text-generation-webui/softprompts")
        os.system(f"ln -s $base_folder/characters text-generation-webui/characters")
        os.system(f"ln -s $base_folder/User .")
    else:
        os.system(f"mkdir text-generation-webui/logs")
        
    os.system(f"ln -s text-generation-webui/logs .")
    os.system(f"ln -s text-generation-webui/characters .")
    os.system(f"ln -s text-generation-webui/models .")
        
    os.system("rm -r sample_data")
    
    os.chdir(main_dir)
    os.system(f"wget https://oobabooga.github.io/settings-colab.json -O settings-colab-template.json")
    os.system(f"rm -rf /content/text-generation-webui/presets/Default.txt /content/text-generation-webui/settings-colab-template.json /content/text-generation-webui/settings-template.json")

    link="https://huggingface.co/Imablank/AnythingV3-1/raw/main/GPT4.txt, https://huggingface.co/Imablank/AnythingV3-1/raw/main/Alpha.json, https://huggingface.co/Imablank/AnythingV3-1/resolve/main/Alpha.png, https://huggingface.co/Imablank/AnythingV3-1/raw/main/settings.json"
    path="/content/text-generation-webui/presets/, /content/characters/,    /content/characters/, /content/text-generation-webui/"
    nan3="GPT4.txt, Alpha.json, Alpha.png, settings.json"; link=link.split(', '); path=path.split(', '); nan3=nan3.split(', ')
    for link, path, nan3 in zip(link, path, nan3):
        universal_download(link, path, nan3)

    os.chdir(main_dir)
    os.system(f"pip install -r requirements.txt && pip install -r extensions/google_translate/requirements.txt && pip install -r extensions/api/requirements.txt")
    
    if model in quant_:
        os.system(f"mkdir /content/text-generation-webui/repositories")
        os.chdir("/repositories/")
        os.system(f"git clone -b v1.0 https://github.com/camenduru/GPTQ-for-LLaMa.git")
        os.chdir("/GPTQ-for-LLaMa/")
        os.system(f"python setup_cuda.py install")

  def prepare_ooba():
    if not os.path.exists(main_dir) or model in quant_ and not os.path.exists("/content/text-generation-webui/repositories"):
      os.chdir("/content")
      os.system(f"git clone https://github.com/oobabooga/text-generation-webui && git clone https://github.com/theubie/complex_memory {main_dir}/extensions/complex_memory")
    
      if old_ooba: os.chdir(main_dir); os.system(f"git checkout f052ab9c8fed3dedc446c3847f10ab22e42bfb37")
      install_ooba()

    else: clear_output(); print("\033[92m\n[ ALREADY INSTALLED -- SKIPPING INSTALL.. ]")
    os.system(f"rm -rf /content/text-generation-webui/models/config.yaml")
    os.chdir("/content")
    clear_output()

class utils:
  from IPython.utils import capture
  from google.colab import files
  from PIL import Image
  import os, sys, base64, subprocess, json, shutil, requests, time, pathlib

  def install_everything(modes):
    pass
    #class oobabooga()
    """
    .prepare_ooba(), install_ooba(), .edgegpt(), .overwrite_profile(), .find_name()
    """
    #def install_silly():
    #def universal_download(link1, path1, nan3):

  def run_silly_tavern():
    p = subprocess.Popen(["lt", "--port", "5001"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    Link=(p.stdout.readline().decode().strip())
    # Get external IP address
    external_ip = requests.get('https://api.ipify.org').text
    external_ip=f"\n###Copy Google Colab's Endpoint IP###\nEndpoint IP: \033[4;92m{external_ip}\033[0m"
    print(external_ip, f"\n###SillyTavern LINK###\n{Link}", sep="\n")
    os.system(f"node server.js")

  def run_ooba():
    global quant_
    params = set()
    if model in quant_: params.add('--wbits 4 --groupsize 128 --model_type Llama')
  
    if load_in_8bit: params.add('--load-in-8bit')
    params.add('--share --chat') if not GetAPI else params.add('--public-api --chat')
    if not text_streaming or activate_google_translate: params.add('--no-stream')
    active_extensions = []
    if activate_sending_pictures: active_extensions.append('send_pictures')
    if activate_character_bias: active_extensions.append('character_bias')
    if complex_memory: active_extensions.append('complex_memory')
    if activate_google_translate: active_extensions.append('google_translate')
    if GetAPI: active_extensions.append('api')
    if GrantBot_Access_to_Bing: active_extensions.append('EdgeGPT')
    active_extensions.append('gallery')
    if len(active_extensions) > 0: params.add(f'--extensions {" ".join(active_extensions)}')
  
    cmd = f"python server.py --verbose --model {b} --settings {main_dir}cache/{_username_} {' '.join(params)}"
    print(cmd)
    os.system(cmd)
  
    #Save to drive system
    if os.path.exists(base_folder):
      username_delete = os.listdir("/content/drive/MyDrive/oobabooga-data/User")
      try: username_delete.remove("pfp_me.png")
      except: pass
      for username_delete in username_delete:
        os.system(f"rm -rf {base_folder}/User/{username_delete}")
      if os.path.exists("/content/text-generation-webui/cache/pfp_me.png"):
        filler=" /content/text-generation-webui/cache/pfp_me.png "
      os.system(f"cp -r /content/text-generation-webui/cache/{_username_}{filler}/content/drive/MyDrive/oobabooga-data/User")

  
  def download_model(type):
    tmp_repo=f"/content/.{huggingface_repo}"
    os.system(f"git lfs install --skip-smudge && export GIT_LFS_SKIP_SMUDGE=1 && git clone https://huggingface.co/{huggingface_org}/{huggingface_repo} /content/.{huggingface_repo} --branch {huggingface_branch}")
    os.system(f"rm -rf  {tmp_repo}/PygmalionCoT-7b-ggml-model-f16.bin {tmp_repo}/PygmalionCoT-7b-ggml-q4_2.bin {tmp_repo}/PygmalionCoT-7b-ggml-q5_1.bin {tmp_repo}/PygmalionCoT-7b-ggml-q8_0.bin {tmp_repo}/PygmalionCoT-7b-4bit-128g.safetensors {tmp_repo}/pytorch_model.bin.index.json {tmp_repo}/training_args.bin {tmp_repo}/trainer_state.json {tmp_repo}/all_results.json {tmp_repo}/eval_results.json {tmp_repo}/.gitattributes {tmp_repo}/train_results.json")
    repo = os.listdir(tmp_repo); clear_output()
    for download in repo:    
      link_path = f"https://huggingface.co/{huggingface_org}/{huggingface_repo}/"
      if ".json" in download or ".txt" in download:
        link_path += f"raw/{huggingface_branch}/{download}"
      else: link_path += f"resolve/{huggingface_branch}/{download}"
      os.system(f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link_path} -d /content/text-generation-webui/models/{type} -o {download}")


In [ ]:
#@title Default title text

#Receive
"""
var_list=sys.argv[1]

#Extract

UserName=var_list[0]
save_logs_to_google_drive=var_list[1]
model=var_list[2]
text_streaming[3]
activate_sending_pictures=var_list[4]
activate_character_bias=var_list[5]
chat_language=var_list[6]
RunWebUI=var_list[7]
GetAPI=var_list[8]
Debug=var_list[9]

activate_google_translate = (chat_language != "English")
"""

#PART 1 INSTALL TextGeN

def universal_download(link1, path1, nan3):
  os.system(f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link1} -d {path1} -o {nan3}")

#SillyTavern
def install_silly():
  ForceInitSteps = []
  class IncrementialInstall:
    def __init__(self, root = "/", tasks = [], force = []):
        self.tasks = tasks
        self.path = os.path.join(root, ".ii")
        self.completed = list(filter(lambda x: not x in force, self.__completed()))
    def __completed(self):
        try:
            with open(self.path) as f:
                return json.load(f)
        except:
            return []
    def addTask(self, name, func):
        self.tasks.append({"name": name, "func": func})
    def run(self):
        todo = list(filter(lambda x: not x["name"] in self.completed, self.tasks))
        try:
            for task in todo:
                task["func"]()
                self.completed.append(task["name"])
        finally:
            with open(self.path, "w") as f:
                json.dump(self.completed, f)
  def create_paths(paths):
    for directory in paths:
        if not os.path.exists(directory):
            os.makedirs(directory)
  def link(srcDir, destDir, files):
    '''
        Link source to dest copying dest to source if not present first
    '''
    for file in files:
        source = os.path.join(srcDir, file)
        dest = os.path.join(destDir, file)
        if not os.path.exists(source):
            os.system(f"cp -r {dest} {source}")
        os.system(f"rm -rf {dest}")
        os.system(f"ln -fs {source} {dest}")
  from google.colab import drive
  if not save_logs_to_google_drive:
    create_paths([
            "/content/SillyTavern-Data"
    ])
  ii = IncrementialInstall(force=ForceInitSteps)
# ---
# SillyTavern py modules
  def cloneTavern():
    os.chdir('/')
    os.system(f"git clone https://github.com/Cohee1207/SillyTavern")
    os.system(f"wget https://huggingface.co/Imablank/AnythingV3-1/resolve/main/ooba/globals.py && wget https://huggingface.co/Imablank/AnythingV3-1/resolve/main/ooba/extras_server.py")
  ii.addTask("Clone SillyTavern", cloneTavern)
  ii.run()
  kargs = ["/content/ckds"]
  kargs += ["--localtunnel", "yes"]

# ---
# nodejs
  os.chdir('/')
  def installNode():
    os.system(f"npm install -g n")
    os.system(f"n 19")
    os.system(f"node --version")
  installNode()
# ---
# TavernAI extras
  import globals
  globals.params = []
  globals.params.append('--cpu')
  os.chdir('/SillyTavern') 
  if save_logs_to_google_drive:
    os.system("env googledrive=2") #%env googledrive=2
    def setupTavernPaths():
        os.chdir('/SillyTavern') 
        tdrive = "/content/drive/MyDrive/SillyTavern"
        create_paths([
                tdrive,
                os.path.join("public", "groups"),
                os.path.join("public", "group chats")
        ])
        link(tdrive, "public", [
                "settings.json",
                "backgrounds",
                "characters",
                "chats",
                "User Avatars",
                "worlds",
                "group chats",
                "groups",
        ])
    ii.addTask("Setup Tavern Paths", setupTavernPaths)
  def installTavernDependencies():
    os.chdir('/SillyTavern') 
    os.system("npm install")
    os.system("npm install -g localtunnel")
  ii.addTask("Install Tavern Dependencies", installTavernDependencies)
  ii.run()

#Oobabooga

class oobabooga:
  global quant_, base_folder, main_dir, cache
  #Main Directories:
  base_folder = "/content/drive/MyDrive/oobabooga-data"
  main_dir = "/content/text-generation-webui/"
  cache = f"{main_dir}cache"

  quant_="GPTQ|4bit|128"
  GrantBot_Access_to_Bing = False
  load_in_8bit = True
  if (model in quant_): load_in_8bit = False
  old_ooba = True
  complex_memory = True
  filler=" "

  def edgegpt():
    import zipfile
    os.chdir("/content/")
    os.system(f"wget https://huggingface.co/Imablank/AnythingV3-1/resolve/main/ooba/EdgeGPT_working.zip")
    with zipfile.ZipFile('/content/EdgeGPT_working.zip', 'r') as zip:
        zip.extractall("/content/text-generation-webui/extensions")
    os.system(f"pip install -r {main_dir}extensions/EdgeGPT/requirements.txt && rm -rf /content/EdgeGPT_working.zip")
    os.chdir(main_dir)
    
  def overwrite_profile():
    global UserName, _username_, base_folder, main_dir, cache
    if not username_found == None and not "" == UserName:
        del_userName= '[{'+username_found+"}]-settings-colab.json"
        os.system(f"rm -rf {cache}/{del_userName}")    
    elif not username_found == None and "" == UserName: _username_exists = "[{"+username_found+"}]-settings-colab.json"; UserName = username_found
    elif username_found == None and "" == UserName: UserName="Anon-san"
    os.chdir(main_dir)  
    j = json.loads(open('settings.json', 'r').read())
    j["google_translate-language string"] = language_codes[chat_language]
    j["name1"] = UserName;
    if " " in UserName: UserName=UserName.replace(" ", "_")
    _username_= '[{'+UserName+"}]-settings-colab.json"
    with open(_username_, 'w') as f:
        f.write(json.dumps(j, indent=4))
    os.system(f"rm -rf {main_dir}cache/{_username_exists}")
    os.system(f"mv /content/text-generation-webui/{_username_} /content/text-generation-webui/cache")
    clear_output()

  def find_name():
    global base_folder
    if os.path.exists(base_folder+"/User"):
        try:
            user_profile = os.listdir(base_folder+"/User")
            for count1 in user_profile:
                os.system(f"cp -r /content/drive/MyDrive/oobabooga-data/User/{count1} /content/text-generation-webui/cache")
        except: pass
    try:
        username_find_dir=os.listdir("/content/text-generation-webui/cache/");
        username_located=", ".join(username_find_dir);
        username_x1=username_located.index("[{")+2;
        username_x2=username_located.index("}]");
        username_found=username_located[username_x1:username_x2]
    except: username_found = None
    finally: return username_found

  def install_ooba():
    global quant_, base_folder, main_dir, cache
    os.system(f"apt-get -y install -qq aria2 && mkdir {main_dir}cache")
    if save_logs_to_google_drive:
        if not os.path.exists(f"{base_folder}"):
            os.mkdir(f"{base_folder}")
        if not os.path.exists(f"{base_folder}/logs"):
            os.mkdir(f"{base_folder}/logs")
        if not os.path.exists(f"{base_folder}/User"):
            os.mkdir(f"{base_folder}/User")
        if not os.path.exists(f"{base_folder}/softprompts"):
            os.mkdir(f"{base_folder}/softprompts")
        if not os.path.exists(f"{base_folder}/characters"):
            shutil.move("text-generation-webui/characters", f"{base_folder}/characters")
        else:
            os.system(f"rm -rf {main_dir}characters")
        
        os.system(f"rm -r text-generation-webui/softprompts")
        os.system(f"ln -s $base_folder/logs text-generation-webui/")
        os.system(f"ln -s $base_folder/softprompts text-generation-webui/softprompts")
        os.system(f"ln -s $base_folder/characters text-generation-webui/characters")
        os.system(f"ln -s $base_folder/User .")
    else:
        os.system(f"mkdir text-generation-webui/logs")
        
    os.system(f"ln -s text-generation-webui/logs .")
    os.system(f"ln -s text-generation-webui/characters .")
    os.system(f"ln -s text-generation-webui/models .")
        
    os.system("rm -r sample_data")
    
    os.chdir(main_dir)
    os.system(f"wget https://oobabooga.github.io/settings-colab.json -O settings-colab-template.json")
    os.system(f"rm -rf /content/text-generation-webui/presets/Default.txt /content/text-generation-webui/settings-colab-template.json /content/text-generation-webui/settings-template.json")

    link="https://huggingface.co/Imablank/AnythingV3-1/raw/main/GPT4.txt, https://huggingface.co/Imablank/AnythingV3-1/raw/main/Alpha.json, https://huggingface.co/Imablank/AnythingV3-1/resolve/main/Alpha.png, https://huggingface.co/Imablank/AnythingV3-1/raw/main/settings.json"
    path="/content/text-generation-webui/presets/, /content/characters/,    /content/characters/, /content/text-generation-webui/"
    nan3="GPT4.txt, Alpha.json, Alpha.png, settings.json"; link=link.split(', '); path=path.split(', '); nan3=nan3.split(', ')
    for link, path, nan3 in zip(link, path, nan3):
        universal_download(link, path, nan3)

    os.chdir(main_dir)
    os.system(f"pip install -r requirements.txt && pip install -r extensions/google_translate/requirements.txt && pip install -r extensions/api/requirements.txt")
    
    if model in quant_:
        os.system(f"mkdir /content/text-generation-webui/repositories")
        os.chdir("/repositories/")
        os.system(f"git clone -b v1.0 https://github.com/camenduru/GPTQ-for-LLaMa.git")
        os.chdir("/GPTQ-for-LLaMa/")
        os.system(f"python setup_cuda.py install")

  if not os.path.exists(main_dir) or model in quant_ and not os.path.exists("/content/text-generation-webui/repositories"):
    os.chdir("/content")
    os.system(f"git clone https://github.com/oobabooga/text-generation-webui && git clone https://github.com/theubie/complex_memory {main_dir}/extensions/complex_memory")
    #install_ooba()
    if old_ooba: os.chdir(main_dir); os.system(f"git checkout f052ab9c8fed3dedc446c3847f10ab22e42bfb37")
  else: clear_output(); print("\033[92m\n[ ALREADY INSTALLED -- SKIPPING INSTALL.. ]")
  os.system(f"rm -rf /content/text-generation-webui/models/config.yaml")
  os.chdir("/content")
  clear_output()




class utils:
  from IPython.utils import capture
  from google.colab import files
  from PIL import Image
  import os, sys, base64, subprocess, json, shutil, requests, time, pathlib
  
  def run_silly_tavern():
    p = subprocess.Popen(["lt", "--port", "5001"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    Link=(p.stdout.readline().decode().strip())
    # Get external IP address
    external_ip = requests.get('https://api.ipify.org').text
    external_ip=f"\n###Copy Google Colab's Endpoint IP###\nEndpoint IP: \033[4;92m{external_ip}\033[0m"
    print(external_ip, f"\n###SillyTavern LINK###\n{Link}", sep="\n")
    os.system(f"node server.js")

  def run_ooba():
    params = set()
    if "Wizard-Vicuna-13B-Uncensored-GPTQ" in model or "4bit" in model: params.add('--wbits 4 --groupsize 128 --model_type Llama')
  
    if load_in_8bit: params.add('--load-in-8bit')
    params.add('--share --chat') if not GetAPI else params.add('--public-api --chat')
    if not text_streaming or activate_google_translate: params.add('--no-stream')
    active_extensions = []
    if activate_sending_pictures: active_extensions.append('send_pictures')
    if activate_character_bias: active_extensions.append('character_bias')
    if complex_memory: active_extensions.append('complex_memory')
    if activate_google_translate: active_extensions.append('google_translate')
    if GetAPI: active_extensions.append('api')
    if GrantBot_Access_to_Bing: active_extensions.append('EdgeGPT')
    active_extensions.append('gallery')
    if len(active_extensions) > 0: params.add(f'--extensions {" ".join(active_extensions)}')
  
    cmd = f"python server.py --verbose --model {b} --settings {main_dir}cache/{_username_} {' '.join(params)}"
    print(cmd)
    os.system(cmd)
  
    #Save to drive system
    if os.path.exists(base_folder):
      username_delete = os.listdir("/content/drive/MyDrive/oobabooga-data/User")
      try: username_delete.remove("pfp_me.png")
      except: pass
      for username_delete in username_delete:
        os.system(f"rm -rf {base_folder}/User/{username_delete}")
      if os.path.exists("/content/text-generation-webui/cache/pfp_me.png"):
        filler=" /content/text-generation-webui/cache/pfp_me.png "
      os.system(f"cp -r /content/text-generation-webui/cache/{_username_}{filler}/content/drive/MyDrive/oobabooga-data/User")

  def download_model(type):
    tmp_repo=f"/content/.{huggingface_repo}"
    os.system(f"git lfs install --skip-smudge && export GIT_LFS_SKIP_SMUDGE=1 && git clone https://huggingface.co/{huggingface_org}/{huggingface_repo} /content/.{huggingface_repo} --branch {huggingface_branch}")
    os.system(f"rm -rf  {tmp_repo}/PygmalionCoT-7b-ggml-model-f16.bin {tmp_repo}/PygmalionCoT-7b-ggml-q4_2.bin {tmp_repo}/PygmalionCoT-7b-ggml-q5_1.bin {tmp_repo}/PygmalionCoT-7b-ggml-q8_0.bin {tmp_repo}/PygmalionCoT-7b-4bit-128g.safetensors {tmp_repo}/pytorch_model.bin.index.json {tmp_repo}/training_args.bin {tmp_repo}/trainer_state.json {tmp_repo}/all_results.json {tmp_repo}/eval_results.json {tmp_repo}/.gitattributes {tmp_repo}/train_results.json")
    repo = os.listdir(tmp_repo); clear_output()
    for download in repo:    
      link_path = f"https://huggingface.co/{huggingface_org}/{huggingface_repo}/"
      if ".json" in download or ".txt" in download:
        link_path += f"raw/{huggingface_branch}/{download}"
      else: link_path += f"resolve/{huggingface_branch}/{download}"
      os.system(f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link_path} -d /content/text-generation-webui/models/{type} -o {download}")

  def models()
    os.chdir(main_dir)
    if "Pygmalion-350m(CPU)" in model:
      huggingface_org="alpindale";huggingface_repo="pygm-350m-experimental";huggingface_branch="main"
    elif "PygmalionCoT-7b" in model:
      huggingface_org="notstoic";huggingface_repo="PygmalionCoT-7b";huggingface_branch="main"
    elif "Wizard-Vicuna-13B-Uncensored-GPTQ" in model:
      huggingface_org="TheBloke";huggingface_repo="Wizard-Vicuna-13B-Uncensored-GPTQ";huggingface_branch= "main"
    elif "Pygmalion_6B_main_Sharded" in model:
      huggingface_org="waifu-workshop";huggingface_repo="pygmalion-6b";huggingface_branch="sharded"
    elif "Pygmalion_6B_original_Sharded" in model:
      huggingface_org="waifu-workshop";huggingface_repo="pygmalion-6b";huggingface_branch="original-sharded"
    elif "Pygmalion_6B_dev_Sharded" in model:
      huggingface_org="waifu-workshop";huggingface_repo="pygmalion-6b";huggingface_branch="dev-sharded"
    elif "Pygmalion-7B" in model:
      huggingface_org="TehVenom";huggingface_repo="Pygmalion-7b-Merged-Safetensors";huggingface_branch="main"
    elif "Metharme-7B" in model:
      huggingface_org="Imablank";huggingface_repo="Metharme-7B-MERGED_WEIGHTS";huggingface_branch="main"
    elif "Pygmalion-13B-4bit" in model:
      huggingface_org="notstoic";huggingface_repo="pygmalion-13b-4bit-128g";huggingface_branch="main"
    elif "Metharme-13B-4bit" in model:
      huggingface_org="TehVenom";huggingface_repo="Metharme-13b-Merged";huggingface_branch="main"


In [ ]:

import sys
from PIL import Image
sys.path.append("/")
import Run_WebUI

from Run_WebUI import *

SyntaxError: ignored

In [ ]:
#@title **⚠️Delete TextGen⚠️**
!rm {main_dir}